In [1]:
import math
import random

import numpy as np
import matplotlib.pyplot as plt

import squigglepy as sq
from squigglepy import bayes
from squigglepy.numbers import K, M, B, T

from copy import copy, deepcopy
from scipy import stats
from pprint import pprint
from tqdm import tqdm
print('Loaded 1')

exec(open('utils.py').read())
print('Loaded 2')

exec(open('modules/tai_timelines.py').read())
print('Loaded TAI timelines module')

exec(open('/Users/peterhurford/dev/forecastflow/library.py').read()) # TODO: Package?
print('Loaded Metaculus module')

Loaded 1
Loaded 2
Loaded TAI timelines module
Loaded Metaculus lib v0.2
Loaded Metaculus module


## AI Timelines Sensitivity Analysis

In [2]:
CURRENT_YEAR = 2023                               # What year to start the run on? (default: 2023)
MAX_YEAR = 2123                                   # What year to end the run on? (default: 2123)
years = list(range(CURRENT_YEAR, MAX_YEAR))

def print_year(y):
    return '>{}'.format(MAX_YEAR) if y > MAX_YEAR else str(int(y))


print('## Default ##')
result = run_tai_model_round(initial_gdp_=23*T,
                             tai_flop_size_=34,
                             nonscaling_delay_=0,
                             algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                             possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                 max_reduction=5,
                                                                                 tai_flop_size=34),
                             initial_flop_per_dollar_=10 ** 18.3,
                             flop_halving_rate_=2.5,
                             max_flop_per_dollar_=10 ** 24,
                             initial_pay_=10 ** 9,
                             gdp_growth_=1.03,
                             max_gdp_frac_=0.01,
                             willingness_ramp_=1,
                             spend_doubling_time_=2.5,
                             p_nonscaling_delay=None,
                             willingness_spend_horizon_=1,
                             print_diagnostic=False)
print('{}: {}'.format('Default', print_year(result)))          

## Default ##
Default: 2045


In [3]:
print('## TAI FLOP Size ##')
for t in range(26, 43):
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=t,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=t),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=5,
                                                                                     tai_flop_size=t),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('FLOP Size for TAI {} log FLOP -> {}'.format(t, print_year(result)))

## TAI FLOP Size ##
FLOP Size for TAI 26 log FLOP -> 2023
FLOP Size for TAI 27 log FLOP -> 2023
FLOP Size for TAI 28 log FLOP -> 2026
FLOP Size for TAI 29 log FLOP -> 2029
FLOP Size for TAI 30 log FLOP -> 2032
FLOP Size for TAI 31 log FLOP -> 2035
FLOP Size for TAI 32 log FLOP -> 2037
FLOP Size for TAI 33 log FLOP -> 2040
FLOP Size for TAI 34 log FLOP -> 2045
FLOP Size for TAI 35 log FLOP -> 2045
FLOP Size for TAI 36 log FLOP -> 2050
FLOP Size for TAI 37 log FLOP -> 2056
FLOP Size for TAI 38 log FLOP -> 2057
FLOP Size for TAI 39 log FLOP -> 2064
FLOP Size for TAI 40 log FLOP -> 2078
FLOP Size for TAI 41 log FLOP -> >2123
FLOP Size for TAI 42 log FLOP -> >2123


In [4]:
print('## GDP Growth ##')
for g in range(0, 5):
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=5,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1 + (g / 100),
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('GDP Growth Rate {}% -> {}'.format(1 + g / 100, print_year(result)))
    

## GDP Growth ##
GDP Growth Rate 1.0% -> 2045
GDP Growth Rate 1.01% -> 2045
GDP Growth Rate 1.02% -> 2045
GDP Growth Rate 1.03% -> 2045
GDP Growth Rate 1.04% -> 2044


In [5]:
print('## Max GDP Frac ##')
for g in [1/(500*K), 1/(250*K), 1/(100*K), 1/(50*K), 1/(10*K), 5/(10*K),
          1/2000, 1/1500, 1/1000, 2/1000, 3/1000, 4/1000, 5/1000, 1/100,
          2/100, 3/100, 4/100]:
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=5,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=g,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('Max GDP Frac {} (1 in ~{:,}) -> {}'.format(round(g, 6), int(round(1 / g)), print_year(result)))
    

## Max GDP Frac ##
Max GDP Frac 2e-06 (1 in ~500,000) -> 2071
Max GDP Frac 4e-06 (1 in ~250,000) -> 2068
Max GDP Frac 1e-05 (1 in ~100,000) -> 2064
Max GDP Frac 2e-05 (1 in ~50,000) -> 2062
Max GDP Frac 0.0001 (1 in ~10,000) -> 2056
Max GDP Frac 0.0005 (1 in ~2,000) -> 2051
Max GDP Frac 0.0005 (1 in ~2,000) -> 2051
Max GDP Frac 0.000667 (1 in ~1,500) -> 2050
Max GDP Frac 0.001 (1 in ~1,000) -> 2049
Max GDP Frac 0.002 (1 in ~500) -> 2047
Max GDP Frac 0.003 (1 in ~333) -> 2046
Max GDP Frac 0.004 (1 in ~250) -> 2046
Max GDP Frac 0.005 (1 in ~200) -> 2045
Max GDP Frac 0.01 (1 in ~100) -> 2045
Max GDP Frac 0.02 (1 in ~50) -> 2044
Max GDP Frac 0.03 (1 in ~33) -> 2044
Max GDP Frac 0.04 (1 in ~25) -> 2044


In [6]:
print('## Spend Doubling Time ##')
for d in range(0, 51, 2):
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=5,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=1 + (d / 10),
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('Spend Doubling Time {}yrs -> {}'.format(1 + d / 10, print_year(result)))
    

## Spend Doubling Time ##
Spend Doubling Time 1.0yrs -> 2042
Spend Doubling Time 1.2yrs -> 2042
Spend Doubling Time 1.4yrs -> 2043
Spend Doubling Time 1.6yrs -> 2043
Spend Doubling Time 1.8yrs -> 2043
Spend Doubling Time 2.0yrs -> 2043
Spend Doubling Time 2.2yrs -> 2044
Spend Doubling Time 2.4yrs -> 2044
Spend Doubling Time 2.6yrs -> 2045
Spend Doubling Time 2.8yrs -> 2045
Spend Doubling Time 3.0yrs -> 2046
Spend Doubling Time 3.2yrs -> 2046
Spend Doubling Time 3.4yrs -> 2047
Spend Doubling Time 3.6yrs -> 2047
Spend Doubling Time 3.8yrs -> 2048
Spend Doubling Time 4.0yrs -> 2048
Spend Doubling Time 4.2yrs -> 2048
Spend Doubling Time 4.4yrs -> 2049
Spend Doubling Time 4.6yrs -> 2049
Spend Doubling Time 4.8yrs -> 2050
Spend Doubling Time 5.0yrs -> 2050
Spend Doubling Time 5.2yrs -> 2050
Spend Doubling Time 5.4yrs -> 2050
Spend Doubling Time 5.6yrs -> 2051
Spend Doubling Time 5.8yrs -> 2051
Spend Doubling Time 6.0yrs -> 2051


In [7]:
print('## Initial FLOP per dollar ##')
for d in [17, 17.5, 18, 18.3, 18.5, 19]:
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=5,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** d,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('Initial log FLOP per dollar {} -> {}'.format(d, print_year(result)))
    

## Initial FLOP per dollar ##
Initial log FLOP per dollar 17 -> 2052
Initial log FLOP per dollar 17.5 -> 2049
Initial log FLOP per dollar 18 -> 2046
Initial log FLOP per dollar 18.3 -> 2045
Initial log FLOP per dollar 18.5 -> 2044
Initial log FLOP per dollar 19 -> 2042


In [8]:
print('## Initial pay ##')
for p in range(70, 101, 2):
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=5,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** (p / 10),
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('Initial pay in log 2022$USD {} (~${}) -> {}'.format(p / 10, numerize(10 ** (p / 10)), print_year(result)))
    

## Initial pay ##
Initial pay in log 2022$USD 7.0 (~$10 million) -> 2051
Initial pay in log 2022$USD 7.2 (~$16 million) -> 2051
Initial pay in log 2022$USD 7.4 (~$25 million) -> 2050
Initial pay in log 2022$USD 7.6 (~$40 million) -> 2049
Initial pay in log 2022$USD 7.8 (~$63 million) -> 2048
Initial pay in log 2022$USD 8.0 (~$100 million) -> 2048
Initial pay in log 2022$USD 8.2 (~$158 million) -> 2047
Initial pay in log 2022$USD 8.4 (~$251 million) -> 2046
Initial pay in log 2022$USD 8.6 (~$398 million) -> 2046
Initial pay in log 2022$USD 8.8 (~$631 million) -> 2045
Initial pay in log 2022$USD 9.0 (~$1 billion) -> 2045
Initial pay in log 2022$USD 9.2 (~$2 billion) -> 2044
Initial pay in log 2022$USD 9.4 (~$3 billion) -> 2044
Initial pay in log 2022$USD 9.6 (~$4 billion) -> 2043
Initial pay in log 2022$USD 9.8 (~$6 billion) -> 2043
Initial pay in log 2022$USD 10.0 (~$10 billion) -> 2043


In [9]:
print('## FLOP halving rate ##')
for f in range(5, 41):
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=5,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=f / 10,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('FLOP halving rate {} -> {}'.format(f / 10, print_year(result)))

    

## FLOP halving rate ##
FLOP halving rate 0.5 -> 2032
FLOP halving rate 0.6 -> 2033
FLOP halving rate 0.7 -> 2034
FLOP halving rate 0.8 -> 2035
FLOP halving rate 0.9 -> 2035
FLOP halving rate 1.0 -> 2036
FLOP halving rate 1.1 -> 2037
FLOP halving rate 1.2 -> 2038
FLOP halving rate 1.3 -> 2038
FLOP halving rate 1.4 -> 2039
FLOP halving rate 1.5 -> 2039
FLOP halving rate 1.6 -> 2040
FLOP halving rate 1.7 -> 2041
FLOP halving rate 1.8 -> 2041
FLOP halving rate 1.9 -> 2042
FLOP halving rate 2.0 -> 2042
FLOP halving rate 2.1 -> 2043
FLOP halving rate 2.2 -> 2043
FLOP halving rate 2.3 -> 2044
FLOP halving rate 2.4 -> 2044
FLOP halving rate 2.5 -> 2045
FLOP halving rate 2.6 -> 2045
FLOP halving rate 2.7 -> 2045
FLOP halving rate 2.8 -> 2046
FLOP halving rate 2.9 -> 2046
FLOP halving rate 3.0 -> 2047
FLOP halving rate 3.1 -> 2047
FLOP halving rate 3.2 -> 2048
FLOP halving rate 3.3 -> 2048
FLOP halving rate 3.4 -> 2049
FLOP halving rate 3.5 -> 2049
FLOP halving rate 3.6 -> 2049
FLOP halving rat

In [10]:
print('## Max FLOP per dollar ##')
for f in range(20, 31):
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=5,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('Max log FLOP per 2022USD$1 {} -> {}'.format(f, print_year(result)))
    

## Max FLOP per dollar ##
Max log FLOP per 2022USD$1 20 -> 2045
Max log FLOP per 2022USD$1 21 -> 2045
Max log FLOP per 2022USD$1 22 -> 2045
Max log FLOP per 2022USD$1 23 -> 2045
Max log FLOP per 2022USD$1 24 -> 2045
Max log FLOP per 2022USD$1 25 -> 2045
Max log FLOP per 2022USD$1 26 -> 2045
Max log FLOP per 2022USD$1 27 -> 2045
Max log FLOP per 2022USD$1 28 -> 2045
Max log FLOP per 2022USD$1 29 -> 2045
Max log FLOP per 2022USD$1 30 -> 2045


In [11]:
print('## Algo Doubling Rate Minimum ##')
for m in [1, 1.3, 1.5, 2, 2.5, 3, 3.5]:
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=m, max_speed=3.5, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=5,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('Algo doubling rate minimum {} -> {}'.format(m, print_year(result)))
    

## Algo Doubling Rate Minimum ##
Algo doubling rate minimum 1 -> 2045
Algo doubling rate minimum 1.3 -> 2045
Algo doubling rate minimum 1.5 -> 2045
Algo doubling rate minimum 2 -> 2045
Algo doubling rate minimum 2.5 -> 2045
Algo doubling rate minimum 3 -> 2045
Algo doubling rate minimum 3.5 -> 2046


In [12]:
print('## Algo Doubling Rate Maximum ##')
for m in [2, 2.5, 3, 3.5, 4, 4.5, 5, 6]:
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=m, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=5,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('Algo doubling rate maximum {} -> {}'.format(m, print_year(result)))

## Algo Doubling Rate Maximum ##
Algo doubling rate maximum 2 -> 2044
Algo doubling rate maximum 2.5 -> 2044
Algo doubling rate maximum 3 -> 2044
Algo doubling rate maximum 3.5 -> 2045
Algo doubling rate maximum 4 -> 2045
Algo doubling rate maximum 4.5 -> 2046
Algo doubling rate maximum 5 -> 2046
Algo doubling rate maximum 6 -> 2048


In [13]:
print('## Possible Algo Reduction minimum ##')
for m in [0, 1, 2, 3, 4, 5]:
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=m,
                                                                                     max_reduction=5,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('Possible algo reduction minimum {} -> {}'.format(m, print_year(result)))

## Possible Algo Reduction minimum ##
Possible algo reduction minimum 0 -> 2056
Possible algo reduction minimum 1 -> 2050
Possible algo reduction minimum 2 -> 2045
Possible algo reduction minimum 3 -> 2043
Possible algo reduction minimum 4 -> 2043
Possible algo reduction minimum 5 -> 2043


In [14]:
print('## Possible Algo Reduction maximum ##')
for m in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=m,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('Possible algo reduction maximum {} -> {}'.format(m, print_year(result)))

## Possible Algo Reduction maximum ##
Possible algo reduction maximum 2 -> 2045
Possible algo reduction maximum 3 -> 2045
Possible algo reduction maximum 4 -> 2045
Possible algo reduction maximum 5 -> 2045
Possible algo reduction maximum 6 -> 2045
Possible algo reduction maximum 7 -> 2045
Possible algo reduction maximum 8 -> 2045
Possible algo reduction maximum 9 -> 2045
Possible algo reduction maximum 10 -> 2045


In [15]:
print('## Willingness ramp ##')
for r in range(1, 11):
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=m,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=r / 10,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=1,
                                 print_diagnostic=False)
    print('Willingness ramp {}x -> {}'.format(round(10 / r, 1), print_year(result)))

## Willingness ramp ##
Willingness ramp 10.0x -> 2040
Willingness ramp 5.0x -> 2042
Willingness ramp 3.3x -> 2042
Willingness ramp 2.5x -> 2043
Willingness ramp 2.0x -> 2043
Willingness ramp 1.7x -> 2044
Willingness ramp 1.4x -> 2044
Willingness ramp 1.2x -> 2044
Willingness ramp 1.1x -> 2044
Willingness ramp 1.0x -> 2045


In [16]:
print('## Willingness spend horizon ##')
for h in range(1, 11):
    result = run_tai_model_round(initial_gdp_=23*T,
                                 tai_flop_size_=34,
                                 nonscaling_delay_=0,
                                 algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                 possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                     max_reduction=m,
                                                                                     tai_flop_size=34),
                                 initial_flop_per_dollar_=10 ** 18.3,
                                 flop_halving_rate_=2.5,
                                 max_flop_per_dollar_=10 ** 24,
                                 initial_pay_=10 ** 9,
                                 gdp_growth_=1.03,
                                 max_gdp_frac_=0.01,
                                 willingness_ramp_=1,
                                 spend_doubling_time_=2.5,
                                 p_nonscaling_delay=None,
                                 willingness_spend_horizon_=h,
                                 print_diagnostic=False)
    print('Willingness spend horizon {}yrs -> {}'.format(h, print_year(result)))

## Willingness spend horizon ##
Willingness spend horizon 1yrs -> 2045
Willingness spend horizon 2yrs -> 2045
Willingness spend horizon 3yrs -> 2045
Willingness spend horizon 4yrs -> 2045
Willingness spend horizon 5yrs -> 2045
Willingness spend horizon 6yrs -> 2045
Willingness spend horizon 7yrs -> 2045
Willingness spend horizon 8yrs -> 2045
Willingness spend horizon 9yrs -> 2045
Willingness spend horizon 10yrs -> 2045


In [17]:
print('## Nonscaling delay 90% -> 10%, delay fixed to 5yrs, change `nonscaling_issue_bottom_year` ##')
years = [2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060]
p_nonscaling_delays = {y: derive_nonscaling_delay_curve(0.9, 0.1, y, verbose=False) for y in tqdm(years)}
for y in years:
    results = [run_tai_model_round(initial_gdp_=23*T,
                                   tai_flop_size_=34,
                                   nonscaling_delay_=5,
                                   algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                   possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                       max_reduction=5,
                                                                                       tai_flop_size=34),
                                   initial_flop_per_dollar_=10 ** 18.3,
                                   flop_halving_rate_=2.5,
                                   max_flop_per_dollar_=10 ** 24,
                                   initial_pay_=10 ** 9,
                                   gdp_growth_=1.03,
                                   max_gdp_frac_=0.01,
                                   willingness_ramp_=1,
                                   spend_doubling_time_=2.5,
                                   p_nonscaling_delay=p_nonscaling_delays[y],
                                   willingness_spend_horizon_=1,
                                   print_diagnostic=False) for _ in tqdm(range(10000))]
    pctiles = sq.get_percentiles(results)
    print('Nonscaling delay bottom year {}: {} (90% CI {} - {})'.format(y,
                                                                        print_year(np.mean(results)),
                                                                        print_year(pctiles[5]),
                                                                        print_year(pctiles[95])))

## Nonscaling delay 90% -> 10%, delay fixed to 5yrs, change `nonscaling_issue_bottom_year` ##


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5091.89it/s]


Nonscaling delay bottom year 2025: 2052 (90% CI 2045 - >2123)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4918.71it/s]


Nonscaling delay bottom year 2030: 2053 (90% CI 2045 - >2123)


...

Nonscaling delay bottom year 2050: 2052 (90% CI 2045 - >2123)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5865.33it/s]


Nonscaling delay bottom year 2055: 2076 (90% CI 2045 - >2123)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5599.15it/s]

Nonscaling delay bottom year 2060: 2053 (90% CI 2045 - >2123)


In [18]:
print('## Nonscaling delay 90% -> 10%, bottom year fixed to 2040, vary delay ##')
p_nonscaling_delay = derive_nonscaling_delay_curve(0.9, 0.1, 2040)
for d in [1, 2, 3, 4, 5, 7, 10, 20, 40]:
    results = [run_tai_model_round(initial_gdp_=23*T,
                                   tai_flop_size_=34,
                                   nonscaling_delay_=d,
                                   algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                   possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                       max_reduction=5,
                                                                                       tai_flop_size=34),
                                   initial_flop_per_dollar_=10 ** 18.3,
                                   flop_halving_rate_=2.5,
                                   max_flop_per_dollar_=10 ** 24,
                                   initial_pay_=10 ** 9,
                                   gdp_growth_=1.03,
                                   max_gdp_frac_=0.01,
                                   willingness_ramp_=1,
                                   spend_doubling_time_=2.5,
                                   p_nonscaling_delay=p_nonscaling_delay,
                                   willingness_spend_horizon_=1,
                                   print_diagnostic=False) for _ in tqdm(range(10000))]
    pctiles = sq.get_percentiles(results)
    print('Nonscaling delay {}yrs: {} (90% CI {} - {})'.format(d,
                                                               print_year(np.mean(results)),
                                                               print_year(pctiles[5]),
                                                               print_year(pctiles[95])))

## Nonscaling delay 90% -> 10%, bottom year fixed to 2040, vary delay ##
|   iter    |  target   |   push    |   shift   |   slope   |
-------------------------------------------------------------
| 3         | -0.08293  | 2.282     | 8.526     | 7.903     |
| 8         | -0.05308  | 0.09665   | 9.39      | 1.412     |
| 50        | -0.03093  | 1.284     | 9.925     | 0.467     |
Curve params found
{'push': 1.284290752713264,
 'shift': 9.92515473141246,
 'slope': 0.46704353428576517}
-


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 6891.34it/s]


Nonscaling delay 1yrs: 2045 (90% CI 2045 - 2050)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 6073.07it/s]


Nonscaling delay 2yrs: 2046 (90% CI 2045 - 2055)


...

Nonscaling delay 10yrs: 2052 (90% CI 2045 - >2123)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 6561.70it/s]


Nonscaling delay 20yrs: 2052 (90% CI 2045 - >2123)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 6573.66it/s]

Nonscaling delay 40yrs: 2053 (90% CI 2045 - >2123)


In [19]:
print('## Nonscaling delay X% -> 10%, bottom year fixed to 2040, delay fixed to 4yrs, vary `initial_chance_of_nonscaling_issue` ##')

ps = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
p_nonscaling_delays = {p: derive_nonscaling_delay_curve(p, 0.1, 2040, verbose=False) for p in tqdm(ps)}
for p in ps:
    results = [run_tai_model_round(initial_gdp_=23*T,
                                   tai_flop_size_=34,
                                   nonscaling_delay_=4,
                                   algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                   possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                       max_reduction=5,
                                                                                       tai_flop_size=34),
                                   initial_flop_per_dollar_=10 ** 18.3,
                                   flop_halving_rate_=2.5,
                                   max_flop_per_dollar_=10 ** 24,
                                   initial_pay_=10 ** 9,
                                   gdp_growth_=1.03,
                                   max_gdp_frac_=0.01,
                                   willingness_ramp_=1,
                                   spend_doubling_time_=2.5,
                                   p_nonscaling_delay=p_nonscaling_delays[p],
                                   willingness_spend_horizon_=1,
                                   print_diagnostic=False) for _ in tqdm(range(10000))]
    pctiles = sq.get_percentiles(results)
    print('Initial chance of nonscaling issue {}%: {} (90% CI {} - {})'.format(int(round(p * 100)),
                                                                               print_year(np.mean(results)),
                                                                               print_year(pctiles[5]),
                                                                               print_year(pctiles[95])))

## Nonscaling delay X% -> 10%, bottom year fixed to 2040, delay fixed to 4yrs, vary `initial_chance_of_nonscaling_issue` ##


  0%|                                                                                                                                                                                | 0/11 [00:00<?, ?it/s]

Data point [0.01 0.01 0.01] is not unique. 1 duplicates registered. Continuing ...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4826.41it/s]


Initial chance of nonscaling issue 10%: 2053 (90% CI 2045 - >2123)


...

Initial chance of nonscaling issue 90%: 2052 (90% CI 2045 - >2123)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5237.71it/s]


Initial chance of nonscaling issue 95%: 2052 (90% CI 2045 - >2123)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4997.87it/s]

Initial chance of nonscaling issue 99%: 2052 (90% CI 2045 - >2123)


In [20]:
print('## Nonscaling delay 90% -> X%, bottom year fixed to 2040, delay fixed to 4yrs, vary `final_chance_of_nonscaling_issue` ##')
ps = [0, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
p_nonscaling_delays = {p: derive_nonscaling_delay_curve(0.9, p, 2040, verbose=False) for p in tqdm(ps)}
for p in ps:
    results = [run_tai_model_round(initial_gdp_=23*T,
                                   tai_flop_size_=34,
                                   nonscaling_delay_=4,
                                   algo_doubling_rate_=algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=34),
                                   possible_algo_reduction_=possible_algo_reduction_fn(min_reduction=2,
                                                                                       max_reduction=5,
                                                                                       tai_flop_size=34),
                                   initial_flop_per_dollar_=10 ** 18.3,
                                   flop_halving_rate_=2.5,
                                   max_flop_per_dollar_=10 ** 24,
                                   initial_pay_=10 ** 9,
                                   gdp_growth_=1.03,
                                   max_gdp_frac_=0.01,
                                   willingness_ramp_=1,
                                   spend_doubling_time_=2.5,
                                   p_nonscaling_delay=p_nonscaling_delays[p],
                                   willingness_spend_horizon_=1,
                                   print_diagnostic=False) for _ in tqdm(range(10000))]
    pctiles = sq.get_percentiles(results)
    print('Final chance of nonscaling issue {}%: {} (90% CI {} - {})'.format(int(round(p * 100)),
                                                                             print_year(np.mean(results)),
                                                                             print_year(pctiles[5]),
                                                                             print_year(pctiles[95])))

## Nonscaling delay 90% -> X%, bottom year fixed to 2040, delay fixed to 4yrs, vary `final_chance_of_nonscaling_issue` ##


 50%|████████████████████████████████████████████████████████████████████████████████████                                                                                    | 6/12 [03:56<04:12, 42.07s/it]

KeyboardInterrupt: 

## Metaculus

In [ ]:
get_question(11558)

In [ ]:
# https://www.metaculus.com/questions/11558/maximum-compute-used-in-ai-training/
# TODO: Fetch from Metaculus, look side by side
# TODO: Be able to predict back
for y in [2022, 2025, 2030]:
    print('-')
    print('## {} ##'.format(y))
    for i in range(3):
        flop_at_max_ = flop_at_max(initial_gdp=variables['initial_gdp'],
                                   gdp_growth=[1.02, 1.025, 1.03][i],
                                   initial_pay=[10*M, 100*M, 1*B][i],
                                   spend_doubling_time=[1, 2, 3][i],
                                   max_gdp_frac=[1/1000, 4/1000, 1/100][i],
                                   initial_flop_per_dollar=10 ** 18,
                                   max_flop_per_dollar=10 ** 24,
                                   flop_halving_rate=[2, 2.5, 3][i],
                                   year=(y - CURRENT_YEAR))

        print('{}: {} max log FLOPs / ~{} petaFLOP/s-days)'.format(['25th', 'mean', '75th'][i],
                                                                   np.round(np.log10(flop_at_max_), 1),
                                                                   log_flop_to_petaflop_sdays(np.log10(flop_at_max_))))
